<a href="https://colab.research.google.com/github/rauana-carvalho/Singularity/blob/main/Final_etl_singularity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ETL - LIMPEZA


In [ ]:
pip install fuzzywuzzy

In [ ]:
import pandas as pd
from google.colab import drive
import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Montar Google Drive
drive.mount('/content/drive')

# Autenticar o usuário com credenciais padrão
creds, _ = default()
gc = gspread.authorize(creds)

def find_similar_columns(columns, threshold=80):
    """
    Encontra colunas semelhantes com base na similaridade de strings.

    Args:
    columns (list): Lista de nomes de colunas.
    threshold (int): Limite de similaridade para considerar duas colunas como semelhantes.

    Returns:
    dict: Dicionário com colunas semelhantes.
    """
    similar_cols_dict = {}
    processed = set()

    for col in columns:
        if col in processed:
            continue
        # Encontrar colunas semelhantes
        matches = process.extract(col, columns, limit=None)
        similar_cols = [match[0] for match in matches if match[1] >= threshold and match[0] != col]
        if similar_cols:
            similar_cols_dict[col] = similar_cols
            processed.update(similar_cols)  # Marcar como processadas

    return similar_cols_dict

def transform_text_to_dataframe(text):
    # A mesma função que você já tem
    lines = text.strip().split('\n')
    data = [line.split(',') for line in lines]

    max_columns = max([len(row) for row in data])
    for i in range(len(data)):
        if len(data[i]) < max_columns:
            data[i].extend([''] * (max_columns - len(data[i])))

    df = pd.DataFrame(data[1:], columns=data[0])
    df = df.loc[:, df.columns.str.strip() != '']
    unified_df = df.groupby(list(df.columns), as_index=False).first()
    return unified_df

def get_unique_or_existing_sheet(sheet, base_name):
    # A mesma função que você já tem
    existing_sheets = [ws.title for ws in sheet.worksheets()]
    if base_name in existing_sheets:
        worksheet = sheet.worksheet(base_name)
    else:
        new_name = base_name
        count = 1
        while new_name in existing_sheets:
            new_name = f"{base_name}_{count}"
            count += 1
        worksheet = sheet.add_worksheet(title=new_name, rows=1000, cols=26)
    return worksheet

def unify_similar_columns(df, similar_columns_dict):
    # A mesma função que você já tem, mas adaptada para usar o dicionário de colunas semelhantes
    for col, similar_cols in similar_columns_dict.items():
        if len(similar_cols) > 0:
            for sim_col in similar_cols:
                if sim_col in df.columns:  # Verificar se a coluna existe
                    df[col] = df[col].combine_first(df[sim_col])
                    df = df.drop(columns=[sim_col])  # Remover a coluna similar após unir
    return df

def process_and_upload_to_sheets(folder_name, sheet_id):
    # Listar arquivos CSV na pasta
    folder_path = f"/content/drive/MyDrive/Singularity/{folder_name}/"
    files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
    print("Arquivos encontrados:", files)

    # Abrir a planilha existente usando o ID
    sh = gc.open_by_key(sheet_id)
    print(f"Planilha com ID '{sheet_id}' já existe e foi aberta.")

    # Lista para armazenar DataFrames
    all_dataframes = []

    # Processar cada arquivo CSV
    for file in files:
        file_path = os.path.join(folder_path, file)

        # Ler o arquivo CSV
        df = pd.read_csv(file_path)
        print(f"Dados do arquivo '{file}' lidos:")
        print(df.head())

        # Limpar nomes de colunas
        df.columns = df.columns.str.strip()
        print("Colunas presentes no DataFrame:", df.columns)

        # Encontrar colunas semelhantes
        similar_columns_dict = find_similar_columns(df.columns.tolist(), threshold=80)
        print("Colunas semelhantes encontradas:", similar_columns_dict)

        # Unificar colunas similares
        df = unify_similar_columns(df, similar_columns_dict)
        print("Dados após unificação de colunas similares:")
        print(df.head())

        # Adicionar o DataFrame unificado à lista
        all_dataframes.append(df)

        # Obter ou criar a aba de acordo com o nome do arquivo
        base_name = file.split('.')[0]
        worksheet = get_unique_or_existing_sheet(sh, base_name)

        # Sobrescrever todos os dados, incluindo os cabeçalhos
        worksheet.clear()
        set_with_dataframe(worksheet, df, row=1, col=1, include_index=False)
        print(f"Dados enviados para a aba: {base_name}")

    # Depois de processar todas as abas, unificar os dados e colar na aba 'Base'
    base_worksheet = get_unique_or_existing_sheet(sh, "Base")

    # Concatenar todos os DataFrames em um único DataFrame
    final_df = pd.concat(all_dataframes, ignore_index=True)

    # Sobrescrever a aba 'Base'
    base_worksheet.clear()
    set_with_dataframe(base_worksheet, final_df, row=1, col=1, include_index=False)
    print("Dados unificados enviados para a aba 'Base'.")

# Nome da pasta do Google Drive onde os arquivos CSV estão armazenados
folder_name = 'Dados brutos'  # Substitua pelo nome da sua pasta

# ID da planilha no Google Sheets (extraído do link da planilha)
sheet_id = '1KI73nAPyJYTz6415SVZfDrCuJwoCJ_Gluv0LKJygCes'  # ID da planilha

# Executar o processo
process_and_upload_to_sheets(folder_name, sheet_id)


ModuleNotFoundError: No module named 'fuzzywuzzy'

## ETL - UNIFICAÇÃO

In [ ]:
import pandas as pd
from google.colab import drive
import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth
import os
import re  # Para usar expressões regulares

# Montar Google Drive
drive.mount('/content/drive')

# Autenticar o usuário
auth.authenticate_user()

# Autorizar com as credenciais padrão do usuário (sem JSON)
creds, _ = default()
gc = gspread.authorize(creds)

def transform_text_to_dataframe(text):
    """
    Transformar texto em um DataFrame do pandas, onde o texto é dividido
    em colunas usando vírgula como delimitador. Campos iguais são unidos,
    mantendo apenas os primeiros valores.
    """
    lines = text.strip().split('\n')
    data = [line.split(',') for line in lines]

    # Ajuste para garantir que o número de colunas seja consistente
    max_columns = max([len(row) for row in data])

    # Preencher linhas que têm menos colunas com valores vazios
    for i in range(len(data)):
        if len(data[i]) < max_columns:
            data[i].extend([''] * (max_columns - len(data[i])))

    # Manter os cabeçalhos originais na primeira linha
    df = pd.DataFrame(data[1:], columns=data[0])

    # Remover colunas que tenham nomes vazios ou nulos
    df = df.loc[:, df.columns.str.strip() != '']

    # Agrupar por todas as colunas (removendo duplicatas)
    unified_df = df.groupby(list(df.columns), as_index=False).first()
    return unified_df

def get_unique_or_existing_sheet(sheet, base_name):
    """
    Retorna uma aba existente ou garante que o nome da nova aba seja único.
    """
    existing_sheets = [ws.title for ws in sheet.worksheets()]

    # Verificar se a aba com o nome base já existe
    if base_name in existing_sheets:
        print(f"Aba '{base_name}' já existe. Usando a aba existente.")
        worksheet = sheet.worksheet(base_name)
    else:
        # Se não existir, criar uma nova aba com um nome único
        new_name = base_name
        count = 1
        while new_name in existing_sheets:
            new_name = f"{base_name}_{count}"
            count += 1
        print(f"Criando nova aba: {new_name}")
        worksheet = sheet.add_worksheet(title=new_name, rows=1000, cols=26)  # Ajuste o número de linhas/colunas conforme necessário

    return worksheet

def unify_similar_columns(df, similar_columns_dict):
    """
    Une colunas similares em uma única coluna, mantendo os primeiros valores.
    """
    for col, similar_cols in similar_columns_dict.items():
        if len(similar_cols) > 1:  # Se houver mais de uma coluna similar
            for sim_col in similar_cols:
                if sim_col in df.columns and sim_col != col:  # Verificar se a coluna existe
                    df[col] = df[col].combine_first(df[sim_col])
                    df = df.drop(columns=[sim_col])  # Remover a coluna similar após unir
    return df

def remove_link_columns(df):
    """
    Remove colunas que contenham links (URLs) nos dados.
    """
    link_pattern = r"http[s]?://"
    cols_with_links = [col for col in df.columns if df[col].astype(str).str.contains(link_pattern).any()]

    if cols_with_links:
        print(f"Removendo colunas que contêm links: {cols_with_links}")
        df = df.drop(columns=cols_with_links)

    return df

def process_and_upload_to_sheets(folder_name, sheet_id):
    # Listar arquivos CSV na pasta
    folder_path = f"/content/drive/MyDrive/Singularity/{folder_name}/"
    files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
    print("Arquivos encontrados:", files)

    # Abrir a planilha existente usando o ID
    sh = gc.open_by_key(sheet_id)
    print(f"Planilha com ID '{sheet_id}' já existe e foi aberta.")

    # Dicionário de colunas similares (substitua pelos nomes reais que você deseja unir)
    similar_columns_dict = {
        'Characteristics: Host Organism': ['Characteristics: Host Organism', 'Host Organism', 'Organism'],
        'Experiment Type': ['Experiment Type', 'Type of Experiment']
        # Adicione mais pares de colunas similares conforme necessário
    }

    # Lista para armazenar DataFrames
    all_dataframes = []

    # Processar cada arquivo CSV
    for file in files:
        file_path = os.path.join(folder_path, file)

        # Ler o arquivo CSV
        df = pd.read_csv(file_path)
        print(f"Dados do arquivo '{file}' lidos:")
        print(df.head())

        # Limpar e ajustar os nomes de colunas (substituir caracteres por underscores)
        df.columns = df.columns.str.replace(r'[.,;: ]+', '_', regex=True)

        # Verificar as colunas disponíveis
        print("Colunas presentes no DataFrame:", df.columns)

        # Adicionar uma coluna com o nome do arquivo
        df.insert(0, 'Source_File', file[:7])  # Adicionar os primeiros 7 dígitos do nome do arquivo

        # Transformar os dados em DataFrame unificado
        unified_df = transform_text_to_dataframe(df.to_csv(index=False))
        print(f"Dados unificados para o arquivo '{file}':")
        print(unified_df.head())

        # Unificar colunas similares
        unified_df = unify_similar_columns(unified_df, similar_columns_dict)
        print("Dados após unificação de colunas similares:")
        print(unified_df.head())

        # Remover colunas com links
        unified_df = remove_link_columns(unified_df)

        # Adicionar o DataFrame unificado à lista
        all_dataframes.append(unified_df)

        # Obter ou criar a aba de acordo com o nome do arquivo
        base_name = file.split('.')[0]
        worksheet = get_unique_or_existing_sheet(sh, base_name)

        # Sobrescrever todos os dados, incluindo os cabeçalhos
        worksheet.clear()

        # Colar os dados a partir da primeira linha
        set_with_dataframe(worksheet, unified_df, row=1, col=1, include_index=False)
        print(f"Dados enviados para a aba: {base_name}")

    # Depois de processar todas as abas, unificar os dados e colar na aba 'Base'
    base_worksheet = get_unique_or_existing_sheet(sh, "Base")

    # Concatenar todos os DataFrames em um único DataFrame
    final_df = pd.concat(all_dataframes, ignore_index=True)

    # Sobrescrever a aba 'Base'
    base_worksheet.clear()
    set_with_dataframe(base_worksheet, final_df, row=1, col=1, include_index=False)
    print("Dados unificados enviados para a aba 'Base'.")

# Nome da pasta do Google Drive onde os arquivos CSV estão armazenados
folder_name = 'Dados brutos'

# ID da planilha no Google Sheets (extraído do link da planilha)
sheet_id = '1KI73nAPyJYTz6415SVZfDrCuJwoCJ_Gluv0LKJygCes'

# Executar o processo
process_and_upload_to_sheets(folder_name, sheet_id)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Arquivos encontrados: ['OSD-665-samples.csv', 'OSD-379-samples.csv']
Planilha com ID '1KI73nAPyJYTz6415SVZfDrCuJwoCJ_Gluv0LKJygCes' já existe e foi aberta.
Dados do arquivo 'OSD-665-samples.csv' lidos:
  Source Name                 Sample Name Characteristics: Organism  \
0    RR-23_F1  RR23_R-EDL_FLT_F1_techrep1              Mus musculus   
1    RR-23_F1  RR23_R-EDL_FLT_F1_techrep2              Mus musculus   
2    RR-23_F2           RR23_R-EDL_FLT_F2              Mus musculus   
3    RR-23_F3           RR23_R-EDL_FLT_F3              Mus musculus   
4    RR-23_F4           RR23_R-EDL_FLT_F4              Mus musculus   

  Characteristics: Strain Characteristics: Animal Source  \
0                C57BL/6J             Jackson Laboratory   
1                C57BL/6J             Jackson Laboratory   
2                C57BL/6J             Jackson Laboratory   
3 

## Chat IA


In [ ]:
!pip install transformers
!pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.2 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Carregar o modelo e o tokenizer do T5
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Função para gerar resposta com o T5
def responder_t5(mensagem, historia):
    # Adicionar a nova mensagem do usuário ao histórico
    historia.append(("Usuário", mensagem))

    # Combinar o histórico em um único contexto
    contexto = " ".join([f"Usuário: {msg}" for _, msg in historia[-5:]])

    # Preparar a entrada para o T5
    input_ids = tokenizer(f"conversa: {contexto}", return_tensors="pt").input_ids

    # Gerar a resposta
    outputs = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)
    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Adicionar a resposta da IA ao histórico
    historia.append(("IA", resposta))

    # Retornar o histórico atualizado
    return historia, historia

# Interface de chat usando o componente Chatbot do Gradio
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat com IA T5", height=400)  # Definir altura diretamente
    msg = gr.Textbox(label="")
    clear = gr.Button("Limpar Chat")

    historia = gr.State([])  # Histórico vazio

    # Função que lida com o envio de mensagem e atualização do chat
    def atualizar_chat(mensagem, historia):
        return responder_t5(mensagem, historia)

    msg.submit(atualizar_chat, inputs=[msg, historia], outputs=[chatbot, historia])
    clear.click(lambda: ([], []), None, [chatbot, historia])

demo.launch()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f42f4313bd38960ef4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## Lógica LLM

In [ ]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 45.0 MB/s eta 0:00:00


In [ ]:
pip install gspread pandas transformers torch scikit-learn


In [ ]:
import gspread
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sklearn.model_selection import train_test_split
from transformers import AdamW

# 1. Autenticar e abrir a planilha (use as credenciais padrão OAuth do gspread)
# Montar Google Drive
drive.mount('/content/drive')

# Autenticar o usuário
auth.authenticate_user()

# Autorizar com as credenciais padrão do usuário (sem JSON)
creds, _ = default()
gc = gspread.authorize(creds)

# Abre a planilha
spreadsheet = gc.open("1KI73nAPyJYTz6415SVZfDrCuJwoCJ_Gluv0LKJygCes")  # Substitua pelo nome da sua planilha do Google Sheets

# Seleciona a primeira aba
sheet = spreadsheet.sheet1  # Ou outra aba se necessário

# Obtém todos os dados da planilha
data = sheet.get_all_records()

# Converte os dados em um DataFrame do pandas
df = pd.DataFrame(data)

# Verifique se os dados foram carregados corretamente
print(df)

# 2. Criar pares de perguntas e respostas a partir dos dados da tabela
qa_pairs = []
for idx, row in df.iterrows():
    qa_pairs.append((f"Qual é o local do experimento com ID {row['ID']}?", row["Local"]))
    qa_pairs.append((f"Qual é a temperatura do experimento com ID {row['ID']}?", row["Temperatura"]))
    qa_pairs.append((f"Qual é o genótipo do experimento com ID {row['ID']}?", row["Genótipo"]))
    qa_pairs.append((f"Qual é o animal usado no experimento com ID {row['ID']}?", row["Animal"]))
    qa_pairs.append((f"Qual é o sexo do animal usado no experimento com ID {row['ID']}?", row["Sexo"]))

# 3. Carregar o modelo e tokenizer T5
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# 4. Função para preparar os dados
def encode_data(pairs):
    inputs = []
    targets = []
    for question, answer in pairs:
        input_ids = tokenizer.encode(f"Pergunta: {question}", return_tensors="pt", max_length=512, truncation=True)
        target_ids = tokenizer.encode(answer, return_tensors="pt", max_length=512, truncation=True)
        inputs.append(input_ids.squeeze(0))  # Remove a dimensão extra
        targets.append(target_ids.squeeze(0))  # Remove a dimensão extra
    return inputs, targets

# Codificar os dados
inputs_combined, targets_combined = encode_data(qa_pairs)

# 5. Divisão em conjunto de treino e teste
train_inputs_combined, test_inputs_combined, train_targets_combined, test_targets_combined = train_test_split(
    inputs_combined, targets_combined, test_size=0.2, random_state=42
)

# 6. Definir otimizador
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3  # Para o exemplo, manter 3 épocas (ajuste conforme necessário)

# 7. Treinamento do modelo
model.train()
for epoch in range(num_epochs):
    print(f"Época {epoch + 1}/{num_epochs}")
    epoch_loss = 0
    for input_ids, target_ids in zip(train_inputs_combined, train_targets_combined):  # Usando o conjunto de treino
        optimizer.zero_grad()  # Zera os gradientes acumulados
        input_ids = input_ids.unsqueeze(0)  # Certificar que são tensores 2D
        target_ids = target_ids.unsqueeze(0)
        outputs = model(input_ids=input_ids, labels=target_ids)  # Passa pelo modelo
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Perda total da época {epoch + 1}: {epoch_loss:.4f}")

# 8. Avaliação no conjunto de teste
model.eval()
with torch.no_grad():
    for input_ids, target_ids in zip(test_inputs_combined, test_targets_combined):
        input_ids = input_ids.unsqueeze(0)
        outputs = model.generate(input_ids=input_ids, max_length=512)
        predicted_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
        actual_answer = tokenizer.decode(target_ids, skip_special_tokens=True)
        print(f"Pergunta: {tokenizer.decode(input_ids[0], skip_special_tokens=True)}")
        print(f"Resposta esperada: {actual_answer}")
        print(f"Resposta predita: {predicted_answer}\n")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Empty DataFrame
Columns: []
Index: []


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
# Função para testar o modelo
def testar_modelo(pergunta):
    model.eval()  # Coloca o modelo em modo de avaliação
    input_ids = tokenizer.encode(f"Pergunta: {pergunta}", return_tensors="pt")

    with torch.no_grad():  # Não calcular gradientes durante o teste
        outputs = model.generate(input_ids, max_length=50)

    resposta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return resposta

# Fazendo uma pergunta para testar o modelo
pergunta_teste = "Qual é o animal do experimento com ID 101?"
resposta = testar_modelo(pergunta_teste)
print(f"Pergunta: {pergunta_teste}")
print(f"Resposta: {resposta}")
